In [2]:
import cv2
import numpy as np
import math
import time
#loading yolo model for object detection 
net = cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")

#getting the hidden layer for the model
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

#getting the class IDs' from the file coco.names for detecting the particular object which is car in this case
with open("coco.names", "r") as file:
    class_list = [line.strip() for line in file.readlines()]
car_label_index = class_list.index("car")

#getting the mask for the colours to be considered
def filter_colors(frame):
    #converting the image from RGB format to HSV for better control of colors
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    #here we are considering both white and yellow colors as the Lanes are usually of these 2 colors

    #defining the yellow color range and mask
    yellow_lower_range = np.array([20, 100, 100])
    yellow_upper_range = np.array([30, 255, 255])
    yellow_mask = cv2.inRange(hsv, yellow_lower_range, yellow_upper_range)

    #defining the white color range and mask
    white_lower_range = np.array([0, 0, 200])
    white_upper_range = np.array([255, 50, 255])
    white_mask = cv2.inRange(hsv, white_lower_range, white_upper_range)

    #combining both the masks
    combined_mask = cv2.bitwise_or(yellow_mask, white_mask)

    # applying the mask to the orginal frame
    filtered_frame = cv2.bitwise_and(frame, frame, mask=combined_mask)

    #uncomment in order to view the intermediate filtered image
    # cv2.imshow("filtered_frame",filtered_frame)
    return filtered_frame

#extracting the edges of the lanes
def edges(frame):
    #converting RGB into LAB format to get the Luminescence 
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    
    # applying CLAHE to the luminescence part so as to enhance the contrast in a certain range of intensity
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    enhanced_lab = cv2.merge((l, a, b))
    enhanced_frame = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)
    
    # Color filtering to isolate white and yellow
    frame = filter_colors(enhanced_frame)
    #uncomment in order to display the yellow and white mask
    # cv2.imshow("yellow and white mask",frame)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #uncomment in order to display the gray image
    # cv2.imshow("gray",gray)

    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    #uncomment in order to display the blurred image
    # cv2.imshow("blur",blur)

    #Using canny edge algorithm to detect the edges
    edges = cv2.Canny(blur, 50, 150)
    #uncomment in order to display the edges image
    # cv2.imshow("edges",edges)
    
    return edges

#defining the region of interest in order to focus only on the Roads.
def ROI(edge_frame):
    height, width = edge_frame.shape[:2]
    #defining the region of interest's dimensions
    vertices = np.array([[
        (50, height),                    # bottom left
        (width - 50, height),            # bottom right
        (width // 2 + 100, height // 2), # top right
        (width // 2 - 100, height // 2)  # top left
    ]], dtype=np.int32)  

    # creating a mask in the same region as that of the roi and applying on the frame to get the masked frame
    mask = np.zeros_like(edge_frame)
    cv2.fillPoly(mask, vertices, 255)
    masked_edges = cv2.bitwise_and(edge_frame, mask)
    #uncomment in order to get the masked edge image
    # cv2.imshow("masked_edges",masked_edges)
    return masked_edges

#getting the top view of the road
def birds_view_func(masked_edges,source_points,destination_points):
    height, width = masked_edges.shape[:2]
    #transformation matrix 
    A = cv2.getPerspectiveTransform(source_points, destination_points)
    top_view = cv2.warpPerspective(masked_edges, A, (width,height), cv2.INTER_LINEAR)
    #uncomment in order to get the top view of lane
    # cv2.imshow("bird's eye's view",top_view)
    return top_view

#collecting the lane pixel points from the top view
def lane_pixels(binary_top_view):
    # collecting the histogram of the bottom half of the top view which is nearest to the car
    histo = np.sum(binary_top_view[binary_top_view.shape[0] // 2:, :], axis=0)

    # mid point of the histogram
    midpoint = int(histo.shape[0] // 2)

    # search window range and kernel size
    margin = 100  
    kernel_size = 50  

    # convolving the kernel with the histogram
    convo_window = np.ones(kernel_size)
    convo_signal = np.convolve(histo, convo_window)

    # redefining the peaks based on convolution signal
    left_peak = np.argmax(convo_signal[:midpoint])
    right_peak = np.argmax(convo_signal[midpoint:]) + midpoint

    # finding the non zero pixels as they represent the lane pixel points
    nonzero = binary_top_view.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])

    # defining the pixels as left lane or right lane depending on the position within the margin
    left_lane_idx = ((nonzero_x >= left_peak - margin) & (nonzero_x < left_peak + margin))
    right_lane_idx = ((nonzero_x >= right_peak - margin) & (nonzero_x < right_peak + margin))

    # extracting the lane pixel positions
    left_x = nonzero_x[left_lane_idx]
    left_y = nonzero_y[left_lane_idx]
    right_x = nonzero_x[right_lane_idx]
    right_y = nonzero_y[right_lane_idx]

    return left_x, left_y, right_x, right_y

#fitting the lane pixels with a 3 degree polynomial 
def fit_lane_pixels(top_view):
    left_x, left_y, right_x, right_y = lane_pixels(top_view)

    # getting the 3 degree polynomial for both the lanes
    left_fit = np.polyfit(left_y, left_x, 3)
    right_fit = np.polyfit(right_y, right_x, 3)

    # getting the y coordinates
    plot_y = np.linspace(0, top_view.shape[0] - 1, top_view.shape[0])

    # getting the x coordinates using coeficients from the polynomial calculated above
    left_fit_x = left_fit[0] * plot_y**3 + left_fit[1] * plot_y**2 + left_fit[2] * plot_y + left_fit[3]
    right_fit_x = right_fit[0] * plot_y**3 + right_fit[1] * plot_y**2 + right_fit[2] * plot_y + right_fit[3]

    return left_fit_x, right_fit_x, plot_y, left_fit, right_fit


#calculating the radius of curvature(ROC) from the 3 degree polynomial 
def calculate_ROC(left_fit, right_fit, plot_y):
    y_eval = np.max(plot_y)
    y_in_m = 30 / 720   # meters per pixel in y dimension

    # curvature formula (given in report)
    left_ROC = ((1 + (3*left_fit[0]*y_eval**2*y_in_m + 2*left_fit[1]*y_eval*y_in_m + left_fit[2])**2)**1.5) / np.abs(6*left_fit[0]*y_eval*y_in_m + 2*left_fit[1])
    right_ROC = ((1 + (3*right_fit[0]*y_eval**2*y_in_m + 2*right_fit[1]*y_eval*y_in_m + right_fit[2])**2)**1.5) /np.abs(6*right_fit[0]*y_eval*y_in_m + 2*right_fit[1])
    avg_ROC = (left_ROC +right_ROC)/2 

    return left_ROC, right_ROC, avg_ROC

#predicting direction of steering
def steering_direction(left_fit_x, right_fit_x, frame_width):
    # getting the center of the road nearest to the car(bottom of the image)
    lane_center = (left_fit_x[-1] + right_fit_x[-1]) / 2
    # assuming vehicle's center is at the center of the frame as most of the cars have their dashcam fitted at the center
    car_center = frame_width / 2  

    # calculating the differnce between the lane center and the car center
    diff = lane_center - car_center

    # determining the direction of steering by setting a threshold of 50 pixels
    if abs(diff) < 50:  
        return "Straight"
    elif diff > 0:
        return "Right"
    else:
        return "Left"

#plotting the lane boundaries on the frame   
def display_lane(original_frame, top_view, left_fit_x, right_fit_x, plot_y, matrix_inv):
    # creating a blank top view and a coloured top view 
    blank_top_view = np.zeros_like(top_view).astype(np.uint8)
    color_top_view = np.dstack((blank_top_view, blank_top_view, blank_top_view))

  
   # defining points for left and right lane boundaries
    points_left = np.array([[x, y] for x, y in zip(left_fit_x, plot_y)])
    points_right = np.array([[x, y] for x, y in zip(right_fit_x, plot_y)])


    #colouring the boundaries 
    cv2.polylines(color_top_view, np.int_([points_left]), isClosed=False, color=(255, 0, 255), thickness=50)
    cv2.polylines(color_top_view, np.int_([points_right]), isClosed=False, color=(255, 0, 255), thickness=50)

    # converting the color image back into the car's perspective using the inverse trasnformation matrix 
    org_frame_plot = cv2.warpPerspective(color_top_view, matrix_inv, (original_frame.shape[1], original_frame.shape[0]))
    
    # combining the lane boundaries with the original frame
    final_lane = cv2.addWeighted(original_frame, 0.7, org_frame_plot, 1, 0)
    return final_lane


#applying all the major lane detecting processes on the input frame
def process_frame(frame):
    edge_frame = edges(frame)
    masked_edges = ROI(edge_frame)
    # source points and destnation points for converting the ROI into top view
    source_points = np.float32([
        [width // 2 - 75, height // 2 + 100],
        [width // 2 + 75, height // 2 + 100],
        [width - 150, height],
        [150, height]
    ])
    destination_points = np.float32([
        [100, 0],
        [width - 100, 0],
        [width - 100, height],
        [100, height]
    ])
    top_view=birds_view_func(masked_edges,source_points,destination_points)
    matrix_inv=cv2.getPerspectiveTransform(destination_points, source_points)
    left_fit_x, right_fit_x, plot_y, left_fit, right_fit = fit_lane_pixels(top_view)
    left_ROC, right_ROC, avg_ROC = calculate_ROC(left_fit, right_fit, plot_y)
    steering = steering_direction(left_fit_x, right_fit_x, frame.shape[1])
    #displaying all the results on the frame
    cv2.putText(frame, f"Left Curvature Radius: {left_ROC:.2f} m", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(frame, f"Right Curvature Radius: {right_ROC:.2f} m", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 200, 255), 2)
    cv2.putText(frame, f"Average Curvature Radius: {avg_ROC:.2f} m", (50, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
    cv2.putText(frame, f"Steering: {steering}", (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 127, 255), 2)
    lane_image = display_lane(frame, top_view, left_fit_x, right_fit_x, plot_y, matrix_inv)
    #uncomment in order to display lane_image
    # cv2.imshow("lane_image",lane_image)
    return lane_image

def car_detection(frame, prev_centers, distance_per_pixel):
    height, width = frame.shape[:2]
    #creating binary large objects for input to YOLO model
    input_blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), (0, 0, 0),True,False)
    net.setInput(input_blob)

    # passing forward to the output layers to get the detections
    detections = net.forward(output_layers)

    # creating the bounding box variable to store the height, width and the center of the boxes,scores for storing the-
    #-probabilities of different class IDs'.
    boxes, scores = [], []
    new_centers = {}

    # iterating over detections
    for detection_layer in detections:
        for obj_data in detection_layer:
            score = obj_data[4]
            prob = obj_data[5:]
            detected_class = np.argmax(prob)
            #if the detected class is the car and its score is greater than 0.5 then we will consider it to be a car
            if detected_class == car_label_index and score > 0.5:
                bbox_center_x, bbox_center_y, bbox_width, bbox_height = (obj_data[:4] * [width, height, width, height]).astype(int)
                top_left_x = int(bbox_center_x - bbox_width / 2)
                top_left_y = int(bbox_center_y - bbox_height / 2)
                boxes.append([top_left_x, top_left_y, bbox_width, bbox_height])
                scores.append(float(prob[detected_class]))
                new_centers[len(new_centers)] = (bbox_center_x, bbox_center_y)

    #applying non maximum supression to discard the overlapping of multiple boxes
    selected_indices = cv2.dnn.NMSBoxes(boxes, scores, score_threshold=0.5, nms_threshold=0.4)
    # flattening it for iterations
    detected_car_ids = set()
    #creating a thereshold of 2100 in order to be considered a near car
    MIN_BBOX_AREA = 2100 #pixel^2
    car_count = 0
    close_car_count=0
    #considering a particular car for calculating its relative speed and area 
    for idx in selected_indices:
        box_x, box_y, box_w, box_h = boxes[idx]
        bbox_area = box_w * box_h
        
        centroid_x, centroid_y = new_centers[idx]
        detected_car_ids.add(idx)

        if idx in prev_centers:
            old_x, old_y = prev_centers[idx]
            #calculating the distance between the successive frames of a particular car
            pixel_shift = ((centroid_x - old_x) ** 2 + (centroid_y - old_y) ** 2) ** 0.5
            meters_shift = pixel_shift * distance_per_pixel
            speed_mps = meters_shift * fps
            speed_kph = speed_mps * 3.6

            # displaying the relative speed of the car calculate above
            cv2.putText(frame, f"Relative_Speed: {speed_kph:.4f} km/h", (box_x, box_y - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        else:
            speed_kph=0
        prev_centers[idx] = (centroid_x, centroid_y)
        #segregating the car as close car or far car 
        if bbox_area>=MIN_BBOX_AREA:
            close_car_count+=1
            color = (0,0,255)
        else:
            color=(0,255,0)

        #bounding the car with a rectangle
        cv2.rectangle(frame, (box_x, box_y), (box_x + box_w, box_y + box_h), color, 2)
    prev_centers = {i: prev_centers[i] for i in detected_car_ids}
    car_count=len(prev_centers)
    return traffic_density(frame,car_count,close_car_count)

def traffic_density(frame,car_count,close_car_count):

    if car_count < 4:
        density_level = "Low"
        density_color = (0, 255, 0)  # green for low traffic
    elif 4 <= car_count < 7:
        density_level = "Medium"
        density_color = (0, 255, 255)  # yellow for medium traffic
    else:
        density_level = "High"
        density_color = (0, 0, 255)  # red for high traffic

    # displaying density info on the frame
    cv2.putText(frame, f"Proximity Count: {close_car_count}", (50, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    cv2.putText(frame, f"Traffic Density: {density_level}", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.7, density_color, 2)

    return frame

#storing the centers of the car of the previous frame
prev_centers = {}

#this could be calibrated depending on the car as well as the camera
#used,for a general case the range should be from 0.02-0.05 hence we assumed it to be 0.04.
distance_per_pixel = 0.04

# getting the input video
input_video = cv2.VideoCapture('curved_lane1.mp4')

# getting the video's FPS and dimensions
fps = input_video.get(cv2.CAP_PROP_FPS)
width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# inititalising the ideowriter with the correct resolution
fourcc = cv2.VideoWriter_fourcc(*'XVID') 

# output files
out = cv2.VideoWriter('output_video.avi', fourcc, 20.0, (width, height)) 
if not out.isOpened() or not out.isOpened():
    print("Error: VideoWriter not opened properly!")
    input_video.release()
    cv2.destroyAllWindows()
    exit()

while input_video.isOpened():
    ret, frame = input_video.read()
    if not ret:
        break

    # processing the frame
    lane_image= process_frame(frame)

    # detecting the cars
    model_frame=car_detection(lane_image, prev_centers, distance_per_pixel)
    out.write(model_frame)

    # showing the output in real time while its saving 
    cv2.imshow('final_output', model_frame)
    # cv2.imshow('final_output', lane_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# releasing everything when completed
input_video.release()
out.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 